<a href="https://colab.research.google.com/github/wtglad/jhu-coursera-making-decisions-in-time/blob/main/Diplo_21k_game_alliance_payoff_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/diplomacy/diplomacy
!pip install diplomacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 2.5 MB/s eta 0:00:00
  Created wheel for diplomacy: filename=diplomacy-1.1.2-py3-none-any.whl size=2339605 sha256=3fe2c5de82fc4390114bd0c790e3704ad5dc44f033cf5e36dd219966d138e471
  Stored in directory: /root/.cache/pip/wheels/8c/9d/b8/a13aeb8754e9731240d55c63022611b3218c18bd93bb43c9b8
Successfully built diplomacy


In [114]:
import random
import os
import pandas as pd
from diplomacy import Game, Map, Power
from diplomacy.engine.renderer import Renderer
import IPython
import networkx as nx
import numpy as np
from progressbar import ProgressBar
import pickle 
import shutil
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Diplomacy/clean-game-history.csv')

In [5]:
len(df['game_id'].unique()) #Should be 20,863

20863

In [19]:
# Cleaning data a bit to standardize formatting
country_aliases = Map().aliases

df['location_alias'] = df[df['location'].notnull()]['location'].apply(lambda x: country_aliases[str(x).upper()
                                .replace('-', ' ')
                                .replace('.', '')
                                .replace('(', '( ')
                                .replace(')', ' )')
                                .replace('GULF OF BOTHNIA', 'GULFOFB')
                                .replace('GULF OF LYONS',  'GULFOFL')
                                ])


df['target_alias'] = df[(df['target'] != '\\N') & (df['target'].notnull())]['target'].apply(lambda x: country_aliases[str(x).upper()
                                .replace('-', ' ')
                                .replace('.', '')
                                .replace('(', '( ')
                                .replace(')', ' )')
                                .replace('GULF OF BOTHNIA', 'GULFOFB')
                                .replace('GULF OF LYONS',  'GULFOFL')
                                .replace('FLEET ', '')
                                .replace('ARMY ', '')
                                .replace(' VIA CONVOY', '')
                                ])

df['target_destination_alias'] = df[(df['target_dest'] != '\\N') & (df['target_dest'].notnull())]['target_dest'].apply(lambda x: country_aliases[str(x).upper()
                                                                                                                          .replace('-', ' ')
                                                                                                                          .replace('.', '')
                                                                                                                          .replace('(', '( ')
                                                                                                                          .replace(')', ' )')
                                                                                                                          .replace('GULF OF BOTHNIA', 'GULFOFB')
                                                                                                                          .replace('GULF OF LYONS',  'GULFOFL')
                                                                                                                          .replace('FLEET ', '')
                                                                                                                          .replace('ARMY ', '')
                                                                                                                          .replace(' VIA CONVOY', '')
                                                                                                                          ])

countries = {
    'E': 'ENGLAND',
    'F': 'FRANCE',
    'G': 'GERMANY',
    'R': 'RUSSIA',
    'I': 'ITALY',
    'A': 'AUSTRIA',
    'T': 'TURKEY'
}

df = df[~df['country'].isnull()]
df['player'] = df['country'].apply(lambda x: countries[x])

In [8]:
def get_target_unit_info(unit_dict, target):
  for power, units in unit_dict.items():
    for unit in units:
      if target in unit:
        return power, unit

In [9]:
def get_power_possible_orders(current_power):
  
  power_possible_orders = []
  
  possible_orders = game.get_all_possible_orders()
  controlled_locations = game.get_orderable_locations(current_power)
  
  for i in controlled_locations:
    power_possible_orders.append(possible_orders[i])

  return power_possible_orders

In [10]:
def create_order(order_type, unit_type, location, target, target_destination, unit_positions):
  if order_type == 'MOVE':
    return (unit_type + ' ' + location + ' - ' + target)
  elif order_type == 'HOLD':
    return (unit_type + ' ' + location + ' H')
  elif order_type == 'SUPPORT':
    power_to_support, unit_to_support = get_target_unit_info(unit_positions, target)  
    
    # Order syntax changes if supporting a movement vs in place
    if target == target_destination:
      return (unit_type + ' ' + location + ' S ' + unit_to_support)
    else:
      return (unit_type + ' ' + location + ' S ' + unit_to_support + ' - ' + target_destination)
  
  elif order_type == 'BUILD':
    return (unit_type +  ' '  + target + ' B')
  elif order_type == 'DESTROY':
    return (unit_type +  ' '  + location + ' D')
  elif order_type == 'RETREAT':
    return (unit_type + ' ' + location + ' R ' + target)
  elif order_type == 'CONVOY':
    power_to_support, unit_to_support = get_target_unit_info(unit_positions, target)
    return (unit_type + ' ' + location + ' C ' + unit_to_support + ' - ' + target_destination)

In [11]:
def process_relationships(df):
  
  relationships = df.groupby(['power', 'power_target']).agg('unit_id').max().reset_index()
  relationships['relationship'] = relationships.apply(lambda x: 'cooperate' if math.isnan(x['unit_id']) else 'attack', axis=1)
  relationships = relationships[['power', 'power_target', 'relationship']]

  return relationships

In [104]:
def create_turn_orders(turn_num, game_df, game):
	
	turn_df = game_df[(game_df['turn_num'] == turn_num)]

	if len(turn_df) > 0: 
		turn_df['order'] = turn_df.apply(lambda x: create_order(x['unit_order'],
								x['type'],
								x['location_alias'],
								x['target_alias'],
								x['target_destination_alias'],
								game.get_state()['units']), axis=1).to_list()


	return turn_df



In [51]:
def compare_orders_with_possible_targets(game, turn_df):

  # Identify all possible player moves
  possible_moves = {}

  for power in list(game.powers.keys()):
    possible_moves[power] = get_power_possible_orders(power)  

  # Parse possible targets from moves
  targets = []

  for power in possible_moves.keys():
    for unit_options in possible_moves[power]:
      for option in unit_options:
        try:
          target = option.split('-')[1]
          if target not in targets:
            targets.append([power, target])
        except Exception:
          continue 

  target_df = pd.DataFrame(targets, columns = ['power', 'target'])
  target_df['target'] = target_df['target'].str.strip()
  target_df = target_df.drop_duplicates()

  # Parse current power holdings
  influences = []

  for power, countries in game.get_state()['influence'].items():
    for country in countries:
      influences.append([country, power])

  influence_df = pd.DataFrame(influences, columns = ['country', 'power'])

  # Compare possible targets to current holdings 
  potential_conflict_df = target_df.merge(influence_df, left_on='target', right_on='country', suffixes=('', '_target'))
  potential_conflict_df = potential_conflict_df[potential_conflict_df['power'] != potential_conflict_df['power_target']]

  #print('potential_conflict_df', potential_conflict_df.columns.tolist())
  #print('turn_df', turn_df.columns.tolist())

  # Evaluate how actual orders compare with potential targets
  actual_conflict_df = potential_conflict_df.merge(turn_df, right_on=['player', 'target_alias'], left_on=['power', 'target'], how='left')

  return actual_conflict_df

In [116]:
pbar = ProgressBar()

for game_id in pbar(df['game_id'].unique()):
  #print(game_id)
  game_df = df[df['game_id'] == game_id].reset_index(drop=True)
  game = Game()

  try:

    for turn_num in range(1, game_df['turn_num'].max() + 1):
      
      #print(turn_num)
      path = '/content/drive/MyDrive/Diplomacy/game-states/' +  str(game_id)

      if not os.path.exists(path):
        os.makedirs(path)

      turn_df = create_turn_orders(turn_num, game_df, game)
      conflict_df = compare_orders_with_possible_targets(game, turn_df)
      
      if len(conflict_df) > 0:
        relationships = process_relationships(conflict_df)
        relationships.to_csv('/content/drive/MyDrive/Diplomacy/game-states/' + str(game_id) + '/' +  str(turn_num) + ' - ' + game.get_current_phase() + ' - relationships.csv', index=False)
      
      # Set orders for each player
      for player in turn_df['player'].unique():
        game.set_orders(player, turn_df[turn_df['player'] == player]['order'].to_list())
      

      with open('/content/drive/MyDrive/Diplomacy/game-states/' + str(game_id) + '/' +  str(turn_num) + ' - ' + game.get_current_phase() + ' - supply_centers.pkl', 'wb') as f:
        pickle.dump(game.get_centers(), f)
      
      '''
      # Uncomment for viz 
      print('Orders')
      game_image = Renderer(game)
      svg = game_image.render(incl_abbrev=True)
      display(IPython.display.SVG(svg))
      '''

      game.process()

    # Final export state / viz
    with open('/content/drive/MyDrive/Diplomacy/game-states/' + str(game_id) + '/' +  str(turn_num) + ' - ' + game.get_current_phase() + ' - supply_centers.pkl', 'wb') as f:
      pickle.dump(game.get_centers(), f)

    '''
    # Uncomment for viz 
    game_image = Renderer(game)
    svg = game_image.render(incl_abbrev=True)
    display(IPython.display.SVG(svg))
    '''
    # Run game analysis
    files = []

    for i in os.listdir('/content/drive/MyDrive/Diplomacy/game-states/' + str(game_id)):
      files.append(['/content/drive/MyDrive/Diplomacy/game-states/' + str(game_id), i])

    file_df = pd.DataFrame(files, columns = ['directory', 'file'])
    game_analysis_files = pd.concat([pd.DataFrame(file_df['file'].apply(lambda x: x.split('-')).values.tolist()), file_df], axis=1)
    game_analysis_files.columns = ['turn', 'phase', 'file', 'parent_dir', 'full_filename']

    game_analysis_files = game_analysis_files.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    relationship_files = game_analysis_files[game_analysis_files['file'] == 'relationships.csv'].reset_index(drop=True)
    relationship_files['turn'] = relationship_files['turn'].astype(int)
    
    supply_center_files = game_analysis_files[game_analysis_files['file'] != 'relationships.csv'].reset_index(drop=True)
    supply_center_files['turn'] = supply_center_files['turn'].astype(int) 
    supply_center_files['lag_turn'] = supply_center_files['turn'] - 1

    rel_sc_map_files = relationship_files.merge(supply_center_files, left_on='turn', right_on='turn', suffixes=('', '_sc_current_turn'))
    rel_sc_map_files = rel_sc_map_files.merge(supply_center_files, left_on='turn', right_on='lag_turn', suffixes=('', '_sc_next_turn'))

    rel_sc_map_files = rel_sc_map_files[[
                                      'turn',
                                      'phase',
                                      'parent_dir',
                                      'full_filename',
                                      'full_filename_sc_current_turn',
                                      'turn_sc_next_turn',
                                      'phase_sc_next_turn',
                                      'full_filename_sc_next_turn'
                                        ]]
    for idx in rel_sc_map_files.index:
    
      row = rel_sc_map_files.iloc[idx]
      
      relationships = pd.read_csv(row['parent_dir'] + '/' + row['full_filename'])
      
      with open(row['parent_dir'] + '/' + row['full_filename_sc_current_turn'], 'rb') as f:
        current_supply_centers = pickle.load(f)

      with open(row['parent_dir'] + '/' + row['full_filename_sc_next_turn'], 'rb') as f:
        next_turn_supply_centers = pickle.load(f)
    
      incremental_supply_centers = []

      for power in current_supply_centers.keys():
        incremental_supply_centers.append([power, len(current_supply_centers[power]), len(next_turn_supply_centers[power])])

      isc_df = pd.DataFrame(incremental_supply_centers, columns = ['power', 'current_scs', 'next_turn_scs'])
      isc_df['payoff'] = isc_df['next_turn_scs'] - isc_df['current_scs']

      relationship_sets = relationships[relationships['relationship'] == 'cooperate'].groupby('power')['power_target'].apply(set).reset_index()
      relationship_sets.columns = ['power', 'cooperates']

      relationship_sets = relationship_sets.merge(relationships[relationships['relationship'] == 'attack'].groupby('power')['power_target'].apply(set).reset_index(), on='power', how='outer')
      relationship_sets.columns = ['power', 'cooperates', 'attacks']

      relationship_sets = relationship_sets.merge(isc_df, on='power', how='outer')
      relationship_sets.to_csv(row['parent_dir'] + '/' + str(row['turn']) + ' - ' + row['phase'] + ' - alliance_attacker_payoffs.csv', index=False)

  except Exception:
    shutil.rmtree('/content/drive/MyDrive/Diplomacy/game-states/' +  str(game_id))
    continue

N/A% (0 of 20863) |                      | Elapsed Time: 0:00:00 ETA:  --:--:--<ipython-input-104-2f3e5d5ecd6a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  turn_df['order'] = turn_df.apply(lambda x: create_order(x['unit_order'],
100% (20863 of 20863) |##################| Elapsed Time: 4:31:47 Time:  4:31:47


In [65]:
game_analysis_

,0,1,2,3,4,directory,file
0,1,S1901M,relationships.csv,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,1 - S1901M - relationships.csv
1,1,S1901M,supply_centers.pkl,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,1 - S1901M - supply_centers.pkl
2,2,F1901M,relationships.csv,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,2 - F1901M - relationships.csv
3,2,F1901M,supply_centers.pkl,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,2 - F1901M - supply_centers.pkl
4,3,W1901A,supply_centers.pkl,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,3 - W1901A - supply_centers.pkl
5,4,S1902M,relationships.csv,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,4 - S1902M - relationships.csv
6,4,S1902M,supply_centers.pkl,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,4 - S1902M - supply_centers.pkl
7,5,S1902R,supply_centers.pkl,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,5 - S1902R - supply_centers.pkl
8,6,F1902M,relationships.csv,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,6 - F1902M - relationships.csv
9,6,F1902M,supply_centers.pkl,None,None,/content/drive/MyDrive/Diplomacy/game-states/1...,6 - F1902M - supply_centers.pkl
